In [159]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.special import gamma
from math import ceil, sqrt

In [160]:
price_df = pd.read_csv('prices.csv')

In [161]:
# Code adapted from https://github.com/BayerSe/RealizedQuantities/blob/master/main.py#L17

def realized_quantity(fun):
    """Applies the function 'fun' to each day separately"""
    return intraday_returns.groupby(pd.Grouper(freq="1d")).apply(fun).loc[index]

# Settings
trading_seconds = 300
avg_sampling_frequency = 300
original_sampling_frequency = 300  # From the process_data.py file

# I don't know what this ratio should be so I'll just set it to one for now
M = trading_seconds / original_sampling_frequency

# Load data and store the intraday returns
data = pd.read_csv("prices.csv")
data['Local time'] = pd.to_datetime(data["Local time"],format='%Y-%m-%d %H:%M:%S')
data = data.set_index("Local time", drop=False)
data = data[["Close_fut"]]
intraday_returns = data.groupby(pd.Grouper(freq="1d")).apply(lambda x: np.log(x / x.shift(1))).dropna()

C:\Users\soren\AppData\Local\Temp\ipykernel_24756\1448499391.py:20: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  intraday_returns = data.groupby(pd.Grouper(freq="1d")).apply(lambda x: np.log(x / x.shift(1))).dropna()


In [162]:
# Index of all days
index = data.groupby(pd.Grouper(freq="1d")).first().dropna().index

In [163]:
# Some constants
mu_1 = np.sqrt((2 / np.pi))
mu_43 = 2 ** (2 / 3) * gamma(7 / 6) * gamma(1 / 2) ** (-1)

In [164]:
# First and last price of each day
prices_open = data.resample('D').first().loc[index]
prices_close = data.resample('D').last().loc[index]

In [165]:
# Return (close-to-close and open-to-close)
r_cc = np.log(prices_close / prices_close.shift(1))
r_oc = np.log(prices_close / prices_open)

In [166]:
# Realized Variance (Andersen and Bollerslev, 1998)
rv = realized_quantity(lambda x: (x ** 2).sum())

# Realized absolute variation (Forsberg and Ghysels, 2007)
rav = mu_1 ** (-1) * M ** (-.5) * realized_quantity(lambda x: x.abs().sum())

# Realized bipower variation (Barndorff-Nielsen and Shephard; 2004, 2006)
bv = mu_1 ** (-2) * realized_quantity(lambda x: (x.abs() * x.shift(1).abs()).sum())

# Standardized tri-power quarticity (see e.g. Forsberg & Ghysels, 2007)
tq = M * mu_43 ** (-3) * realized_quantity(
    lambda x: (x.abs() ** (4 / 3) * x.shift(1).abs() ** (4 / 3) * x.shift(2).abs() ** (4 / 3)).sum())

# Jump test by Huang and Tauchen (2005)
j = (np.log(rv+0.00000001) - np.log(bv+0.00000001)) / \
    ((mu_1 ** -4 + 2 * mu_1 ** -2 - 5) / (M * tq * bv ** -2)) ** 0.5
jump = j.abs() >= stats.norm.ppf(0.999)

In [167]:
# Separate continuous and discontinuous parts of the quadratic variation
iv = pd.Series(0, index=index)

In [110]:
# Can't employ these since no jumps were detected
#iv[jump["Close_fut"].values.squeeze()] = bv[jump["Close_fut"].values.squeeze()] ** 0.5
#iv[not jump["Close_fut"].values] = rv[not jump["Close_fut"].values] ** 0.5

# jv = pd.Series(0, index=index)
# jv[jump] = rv[jump] ** 0.5 - bv[jump] ** 0.5
# jv[jv < 0] = 0

In [168]:
# Realized Semivariance (Barndorff-Nielsen, Kinnebrock and Shephard, 2010)
rv_m = realized_quantity(lambda x: (x ** 2 * (x < 0)).sum())
rv_p = realized_quantity(lambda x: (x ** 2 * (x > 0)).sum())

In [169]:
# Signed jump variation (Patton and Sheppard, 2015)
sjv = rv_p - rv_m
sjv_p = sjv * (sjv > 0)
sjv_m = sjv * (sjv < 0)

# Realized Skewness and Kurtosis  (see, e.g. Amaya, Christoffersen, Jacobs and Vasquez, 2015)
rm3 = realized_quantity(lambda x: (x ** 3).sum())
rm4 = realized_quantity(lambda x: (x ** 4).sum())
rs = np.sqrt(M) * rm3 / rv ** (3 / 2)
rk = M * rm4 / rv ** 2

Export all the features we made to a df

In [170]:
# Export data
out = pd.concat([r_cc, r_oc, rav, rv ** .5, bv ** .5, rv_m ** 0.5, rv_p ** 0.5,
                    sjv, sjv_p, sjv_m, rs, rk], axis=1)
out.columns = ['r_cc', 'r_oc', 'rav', 'rvol', 'bvol', 'rvol_m', 'rvol_p',
                 'sjv', 'sjv_p', 'sjv_m', 'rs', 'rk']
out.to_csv('realized_quantities.csv')

Okay, so let's try to make some more features

In [157]:
# code from https://gist.github.com/linuskohl/690da335a34ebf1cfc5ab27973e16ee5
def movmean(v, kb, kf):
    """
    Computes the mean with a window of length kb+kf+1 that includes the element 
    in the current position, kb elements backward, and kf elements forward.
    Nonexisting elements at the edges get substituted with NaN.
    Args:
        v (list(float)): List of values.
        kb (int): Number of elements to include before current position
        kf (int): Number of elements to include after current position
    Returns:
        list(float): List of the same size as v containing the mean values
    """
    m = len(v) * [np.nan]
    for i in range(kb, len(v)-kf):
        m[i] = np.mean(v[i-kb:i+kf+1])
    return m

def LeeMykland(df, sampling, significance_level=0.01):
    """
    "Jumps in Equilibrium Prices and Market Microstructure Noise"
    - by Suzanne S. Lee and Per A. Mykland
    
    "https://galton.uchicago.edu/~mykland/paperlinks/LeeMykland-2535.pdf"
    
    Args:
        S (list(float)): An array containing prices, where each entry 
                         corresponds to the price sampled every 'sampling' minutes.
        sampling (int): Minutes between entries in S
        significance_level (float): Defaults to 1% (0.001)
        
    Returns:
        A pandas dataframe containing a row covering the interval 
        [t_i, t_i+sampling] containing the following values:
        J:   Binary value is jump with direction (sign)
        L:   L statistics
        T:   Test statistics
        sig: Volatility estimate
    """
    #tm = 252*24*60 # Trading minutes
    # Sincere we are only calculating within each day
    tm = 107 * 5
    S = df['Close_fut'].values
    k   = ceil(sqrt(tm/sampling))
    r = np.append(np.nan, np.diff(np.log(S)))
    bpv = np.multiply(np.absolute(r[:]), np.absolute(np.append(np.nan, r[:-1])))
    bpv = np.append(np.nan, bpv[0:-1]).reshape(-1,1) # Realized bipower variation
    sig = np.sqrt(movmean(bpv, k-3, 0)) # Volatility estimate
    L   = r/sig
    n   = np.size(S) # Length of S
    c   = (2/np.pi)**0.5
    Sn  = c*(2*np.log(n))**0.5
    Cn  = (2*np.log(n))**0.5/c - np.log(np.pi*np.log(n))/(2*c*(2*np.log(n))**0.5)
    beta_star   = -np.log(-np.log(1-significance_level)) # Jump threshold
    T   = (abs(L)-Cn)*Sn
    J   = (T > beta_star).astype(float)
    J   = J*np.sign(r) # Add direction
    # First k rows are NaN involved in bipower variation estimation are set to NaN.
    J[0:k] = np.nan
    # Build and retunr result dataframe
    return pd.DataFrame({'L': L,'sig': sig, 'T': T,'J':J})

def curried_Lee(x):
    return LeeMykland(x, sampling=5)

# a = realized_quantity(curried_Lee)
# a[a[['L', 'sig', 'T',"J"]].notnull().all(1)]

# not using the above approach for now since it gives mostly NaNs
# its very likely I'm doing something wrong and it could actually work